# Optimizing Revenue through Price Point Selection for Microtransactions

**Objective:** The core objective of this project is to harness the power of A/B testing for determining the most effective price point for a newly launched feature on the FarmBurg platform. This determination is crucial for ensuring the profitability and sustainability of the feature.

Brian, the proprietor of FarmBurg, has implemented an A/B test involving three distinct groups, denoted as 'A', 'B', and 'C'. Each group was presented with a different price point for the new feature, \\$0.99, \\$1.99, and \\$4.99 respectively.

> Note: during the initial stages of my analysis, I investigate the purchasing behaviors across the groups with the underlying assumption that all groups were presented with the same price. This detail is considered later, particularly when I conduct the chi-square test and determine the appropriate hypothesis tests to utilize at each step of my analysis afterwards.

My aim is to ensure that this new feature contributes at least \\$1,000 weekly towards the platform's revenue. Brian, the owner of the platform, has specified this amount as the baseline revenue needed to cover the costs associated with the feature's development and continuous maintenance

Data: The dataset, 'clicks.csv', includes:

- `user_id`: a unique id for each visitor
- `group`: the group to which the visitor was assigned (A, B, or C)
- `is_purchase`: whether the visitor made a purchase (Yes or No)

**Outcome:** Based on the analysis, which price point should Brian choose for the new feature to maximize the chances of meeting or exceeding the target weekly revenue?'

---

## 1. Data Loading and Initial Inspection

The first step of my analysis is to load the provided data and perform an initial inspection. The data comes from a CSV file named clicks.csv and includes the following columns:

    `user_id`: A unique id for each visitor to the FarmBurg site.
    `group`: Indicates the group (A, B, or C) to which the visitor was assigned.
    `is_purchase`: Specifies whether the visitor made a purchase (Yes) or not (No).

I will load this data into a pandas DataFrame and use the head() method to display the first few rows. This will give a sense of the data structure and allow me to confirm that the data has been loaded correctly.

> Note: I am using the pandas library to load and manipulate the data. The numpy library will be used for numerical operations, while the scipy.stats library will be used for statistical tests.

Question: Are all sample groups of equal size? Let's verify this by counting the number of users in each group.

In [7]:
# Import necessary libraries
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, binomtest

# Load data
abdata = pd.read_csv('clicks.csv')

# Preview data
abdata.head()

,user_id,group,is_purchase
0,8e27bf9a,A,No
1,eb89e6f0,A,No
2,7119106a,A,No
3,e53781ff,A,No
4,02d48cf1,A,Yes


First, I will inspect the balance of the groups to ensure even distribution.

In [6]:
abdata['group'].value_counts()

A    1666
B    1666
C    1666
Name: group, dtype: int64

## 2. Chi-Square Test of Independence 

My null hypothesis is that there is no association between group assignment and purchase likelihood, i.e., the purchase rates are the same in all groups. The alternative hypothesis is that there is an association, i.e., the purchase rate in at least one group is different from the others.

I will save the p-value from the Chi-square test in a variable named pval and print the result. I will use a significance threshold (α) of 0.05 to decide whether to reject the null hypothesis.

If the p-value ≤ 0.05, I will reject the null hypothesis and conclude that there is a significant difference in the purchase rate across the groups. If the p-value > 0.05, I will not reject the null hypothesis and conclude that there is not enough evidence to suggest a difference in purchase rates.

> Note: I will use the pd.crosstab() function to generate this table, and the chi2_contingency() function from the scipy.stats module to perform the Chi-square test.

In [21]:
# Create contingency table
Xtab = pd.crosstab(abdata['group'], abdata['is_purchase'])

# Chi-square test of independence
chi2, p_val, dof, expected = chi2_contingency(Xtab)

# Print result
print(f"p-value = {p_val}, Significant? {'Yes' if p_val < 0.05 else 'No'}")

p-value = 2.4126213546684264e-35, Significant? Yes


**The p-value of 2.4126213546684264e-35 is extremely small, thus I can conclude that there is a significant difference between the purchase rate for each group.**

## 3. Reporintg Back to Brian

After concluding the Chi-square test, I decided to discuss my findings with our stakeholder, Brian.

Our Conversation with Brian:

Me: *"Hi Brian, can you tell me more about the test you were running?"*

Brian: *"Sure, we are attempting to encourage users to purchase a small FarmBurg upgrade package. It's referred to as a microtransaction. We're unsure about the optimal pricing for it, so we've tested three different price points: \\$0.99 for group 'A', \\$1.99 for group 'B', and \\$4.99 for group 'C'. From our tests, it appears that significantly more people bought the upgrade package at \\$0.99, so that's the price we're considering."*

> #### **It seems I acted too soon with the Chi-Square test.** It wasn't necessarily the best fit for this problem. 
While it's true that more people might be inclined to purchase the upgrade at $0.99, this doesn't necessarily equate to higher revenue. What I really need to determine is whether each price point will generate enough revenue to meet Brians target. 

Me: *"Brian, do you have an idea of how much revenue you'd need to generate to justify this feature?"*

Brian: *"I'd estimate that we'd need to make a minimum of $1000 per week from this feature to justify its development and maintenance."*

### That gives me a clearer target to aim for.

---

## 4. Calculating Purchase Rates and Revenues

To justify the cost of this feature, I must calculate the necessary purchase rate at each price point. Starting with the total weekly visitors on the site, we know Brian ran his original test over a week, implying the number of visitors in our dataset represents a typical week's visitor count.

First, I will calculate the total number of site visitors from the data and save the value in a variable named `num_visits`.

In [13]:
num_visits = len(abdata)
print(num_visits)

4998


I know the number of weekly visitors (`num_visits`). Now, I need to calculate the number of visitors who would need to purchase the upgrade package at each price point (\\$0.99, \\$1.99, \\$4.99) to generate Brian’s minimum revenue target of \\$1,000 per week.

Let's calculate the number of sales that would be needed to reach \\$1,000 of revenue at each price point and print out the results.

In [22]:
target = 1000
num_sales_needed_099 = np.ceil(target/0.99)
num_sales_needed_199 = np.ceil(target/1.99)
num_sales_needed_499 = np.ceil(target/4.99)

num_sales_needed_099,num_sales_needed_199,num_sales_needed_499

(1011.0, 503.0, 201.0)

Knowing the required number of sales, I can calculate the proportion of weekly visitors who would need to make a purchase in order to meet that goal.

In [15]:
p_sales_needed_099 = num_sales_needed_099/num_visits
p_sales_needed_199 = num_sales_needed_199/num_visits
p_sales_needed_499 = num_sales_needed_499/num_visits

print(f"The proportion of weekly visitors to break even if Brian prices the product at $0.99 is {p_sales_needed_099}")
print(f"The proportion of weekly visitors to break even if Brian prices the product at $1.99 is {p_sales_needed_199}")
print(f"The proportion of weekly visitors to break even if Brian prices the product at $4.99 is {p_sales_needed_499}")

The proportion of weekly visitors to break even if Brian prices the product at $0.99 is 0.20228091236494597
The proportion of weekly visitors to break even if Brian prices the product at $1.99 is 0.10064025610244097
The proportion of weekly visitors to break even if Brian prices the product at $4.99 is 0.040216086434573826


Now I am ready to revisit Brian's question. 

I wants to know if the percent of any Group A/B/C, that purchased an upgrade, is significantly greater than the proportion of weekly visitors needed to achieve \\$1000 in revenue per week - measure by `p_sales_needed_099`. To answer this, I can focus on just the visitors in group A, and compare the number of purchases to `p_sales_needed_099`. And then repeat for the following two Groups.

Given this scenario, a suitable test to analyze the purchase rates for each price point group is a binomial test.

In [23]:
from scipy.stats import binomtest

# Perform binomial test for group A ($0.99)
samp_size_099 = Xtab.loc['A']['No'] + Xtab.loc['A']['Yes']
sales_099 = Xtab.loc['A']['Yes']
pvalueA = binomtest(sales_099,samp_size_099,p_sales_needed_099,'greater')

# Perform binomial test for group B ($1.99)
samp_size_199 = Xtab.loc['B']['No'] + Xtab.loc['B']['Yes']
sales_199 = Xtab.loc['B']['Yes']
pvalueB = binomtest(sales_199,samp_size_199,p_sales_needed_199,'greater')

# Perform binomial test for group C ($4.99)
samp_size_499 = Xtab.loc['C']['No'] + Xtab.loc['C']['Yes']
sales_499 = Xtab.loc['C']['Yes']
pvalueC = binomtest(sales_499,samp_size_499,p_sales_needed_499,'greater')

# Print the p-values
print(pvalueA)
print(pvalueB)
print(pvalueC)


BinomTestResult(k=316, n=1666, alternative='greater', statistic=0.18967587034813926, pvalue=0.9058887362654593)
BinomTestResult(k=183, n=1666, alternative='greater', statistic=0.10984393757503001, pvalue=0.11441815431122217)
BinomTestResult(k=83, n=1666, alternative='greater', statistic=0.04981992797118848, pvalue=0.029642608610084057)


In [20]:
pvalueA = binomtest(sales_099,samp_size_099,p_sales_needed_099,'greater')
pvalueA

BinomTestResult(k=316, n=1666, alternative='greater', statistic=0.18967587034813926, pvalue=0.9058887362654593)

In [62]:
pvalueB = binomtest(sales_199,samp_size_199,p_sales_needed_199,'greater')
pvalueB

BinomTestResult(k=183, n=1666, alternative='greater', statistic=0.10984393757503001, pvalue=0.11441815431122217)

In [24]:
pvalueC = binomtest(sales_499,samp_size_499,p_sales_needed_499,'greater')
pvalueC

BinomTestResult(k=83, n=1666, alternative='greater', statistic=0.04981992797118848, pvalue=0.029642608610084057)

## 5. Final Recommendation

Based on the results of the binomial tests conducted for each price point group and considering a significance threshold of 0.05, I can determine the suitable price for Brian to charge for the upgrade package.

My recommendation:

Among the three price points tested, only the \\$4.99 price point (Group C) shows a significantly higher purchase rate compared to the target. This suggests that setting the price at \\$4.99 would be the most profitable option for Brian.

**Therefore, I recommend Brian to charge \\$4.99 for the upgrade package based on the observed sales rate and the goal of generating sufficient revenue.**